In [1]:
import os
import sys

spark_path = "/opt/spark/"

os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = spark_path
os.environ['SPARK_CLASSPATH'] = "/home/ubuntu/postgresql-9.3-1103-jdbc41.jar"

sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.3-src.zip")

In [2]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext, Row

In [3]:
sc = SparkContext("local", "test")

In [4]:
sc

In [5]:
sqlctx = SQLContext(sc)

In [6]:
sqlctx

In [7]:
properties = {
    "user": "clientdemo",
    "password": "clientdemo4"
}

In [8]:
url="jdbc:postgresql://postgresdemo.c3yxphqgag3s.ap-southeast-1.rds.amazonaws.com:5432/postgresdemo"

In [9]:
df = sqlctx.read.jdbc(url=url, table="trans", properties=properties)

In [10]:
df.take(5)

[Row(userid=u'3893506', quantity=0, product_id=u'com.gopaktor.subscription.v1.premium.1m', purchase_date=datetime.datetime(2016, 9, 7, 13, 59, 45), platform=u'ios', revenue=31.28, renewal=0, insider=0, receipt_id=u'1984955'),
 Row(userid=u'3793254', quantity=0, product_id=u'com.gopaktor.subscription.v1.premium.1m', purchase_date=datetime.datetime(2016, 9, 25, 3, 6, 11), platform=u'ios', revenue=31.28, renewal=0, insider=0, receipt_id=u'2170195'),
 Row(userid=u'6148794', quantity=0, product_id=u'com.gopaktor.subscription.v1.premium.1m', purchase_date=datetime.datetime(2016, 10, 1, 2, 27, 37), platform=u'ios', revenue=31.28, renewal=0, insider=0, receipt_id=u'2259995'),
 Row(userid=u'6149220', quantity=0, product_id=u'com.gopaktor.subscription.v4.premium.1m', purchase_date=datetime.datetime(2016, 10, 1, 6, 39, 16), platform=u'android', revenue=31.28, renewal=0, insider=0, receipt_id=u'1980761'),
 Row(userid=u'6149432', quantity=0, product_id=u'com.gopaktor.subscription.v4.premium.1m', pu

In [11]:
data = df.collect()

In [12]:
import pandas as pd

In [13]:
data_df = pd.DataFrame(data, columns=df.columns)

In [14]:
data_df.head()

userid  quantity                               product_id  \
0  3893506         0  com.gopaktor.subscription.v1.premium.1m   
1  3793254         0  com.gopaktor.subscription.v1.premium.1m   
2  6148794         0  com.gopaktor.subscription.v1.premium.1m   
3  6149220         0  com.gopaktor.subscription.v4.premium.1m   
4  6149432         0  com.gopaktor.subscription.v4.premium.1m   

        purchase_date platform  revenue  renewal  insider receipt_id  
0 2016-09-07 13:59:45      ios    31.28        0        0    1984955  
1 2016-09-25 03:06:11      ios    31.28        0        0    2170195  
2 2016-10-01 02:27:37      ios    31.28        0        0    2259995  
3 2016-10-01 06:39:16  android    31.28        0        0    1980761  
4 2016-10-01 09:07:49  android    31.28        0        0    1980865

In [15]:
import yaml
with open('/home/ubuntu/MappingBuffer.yml', 'r') as f:
    doc = yaml.load(f)

In [16]:
doc['column_map']['TRANSACTION_MASTER']

{'cust_id': 'userid',
 'product_id': 'product_id',
 'quantity': 'quantity',
 'renewal': 'renewal',
 'revenue': 'revenue',
 'timestamp': 'purchase_date'}

In [17]:
data_df.rename(columns={doc['column_map']['TRANSACTION_MASTER']['cust_id']:'cust_id'}, inplace = True)
data_df.rename(columns={doc['column_map']['TRANSACTION_MASTER']['revenue']:'revenue'}, inplace = True)
data_df.rename(columns={doc['column_map']['TRANSACTION_MASTER']['product_id']:'product_id'}, inplace = True)
data_df.rename(columns={doc['column_map']['TRANSACTION_MASTER']['timestamp']:'timestamp'}, inplace = True)

In [18]:
data_df.head()

cust_id  quantity                               product_id  \
0  3893506         0  com.gopaktor.subscription.v1.premium.1m   
1  3793254         0  com.gopaktor.subscription.v1.premium.1m   
2  6148794         0  com.gopaktor.subscription.v1.premium.1m   
3  6149220         0  com.gopaktor.subscription.v4.premium.1m   
4  6149432         0  com.gopaktor.subscription.v4.premium.1m   

            timestamp platform  revenue  renewal  insider receipt_id  
0 2016-09-07 13:59:45      ios    31.28        0        0    1984955  
1 2016-09-25 03:06:11      ios    31.28        0        0    2170195  
2 2016-10-01 02:27:37      ios    31.28        0        0    2259995  
3 2016-10-01 06:39:16  android    31.28        0        0    1980761  
4 2016-10-01 09:07:49  android    31.28        0        0    1980865

In [19]:
cust_prod = data_df[['cust_id', 'product_id']]

In [20]:
cust_prod.head()

cust_id                               product_id
0  3893506  com.gopaktor.subscription.v1.premium.1m
1  3793254  com.gopaktor.subscription.v1.premium.1m
2  6148794  com.gopaktor.subscription.v1.premium.1m
3  6149220  com.gopaktor.subscription.v4.premium.1m
4  6149432  com.gopaktor.subscription.v4.premium.1m

In [21]:
uniq_prod = pd.DataFrame(cust_prod.product_id.unique())

In [22]:
uniq_prod

0
0        com.gopaktor.subscription.v1.premium.1m
1        com.gopaktor.subscription.v4.premium.1m
2        com.gopaktor.subscription.v3.premium.6m
3                    com.gopaktor.credit.v2.1000
4        com.gopaktor.subscription.v3.premium.3m
5        com.gopaktor.subscription.v2.premium.3m
6        com.gopaktor.subscription.v2.premium.6m
7   com.gopaktor.subscription.v2.premium.1m.3000
8       com.gopaktor.subscription.v11.premium.6m
9       com.gopaktor.subscription.v12.premium.1m
10  com.gopaktor.subscription.v2.premium.1m.2000
11  com.gopaktor.subscription.v2.premium.3m.2000
12      com.gopaktor.subscription.v12.premium.3m
13  com.gopaktor.subscription.v2.premium.6m.2000
14                  com.gopaktor.credit.v2.10000
15                   com.gopaktor.credit.v2.3000
16                   com.gopaktor.credit.v2.6000
17                   com.gopaktor.credit.v3.2000

In [23]:
uniq_prod.rename(columns={0:doc['column_map']['TRANSACTION_MASTER']['product_id']}, inplace = True)

In [24]:
uniq_prod.head()

product_id
0  com.gopaktor.subscription.v1.premium.1m
1  com.gopaktor.subscription.v4.premium.1m
2  com.gopaktor.subscription.v3.premium.6m
3              com.gopaktor.credit.v2.1000
4  com.gopaktor.subscription.v3.premium.3m

In [25]:
uniq_prod['product_id'] = uniq_prod['product_id'].astype('category')

In [26]:
uniq_prod['code'] = uniq_prod['product_id'].cat.codes

In [27]:
uniq_prod

product_id  code
0        com.gopaktor.subscription.v1.premium.1m     5
1        com.gopaktor.subscription.v4.premium.1m    17
2        com.gopaktor.subscription.v3.premium.6m    16
3                    com.gopaktor.credit.v2.1000     0
4        com.gopaktor.subscription.v3.premium.3m    15
5        com.gopaktor.subscription.v2.premium.3m    11
6        com.gopaktor.subscription.v2.premium.6m    13
7   com.gopaktor.subscription.v2.premium.1m.3000    10
8       com.gopaktor.subscription.v11.premium.6m     6
9       com.gopaktor.subscription.v12.premium.1m     7
10  com.gopaktor.subscription.v2.premium.1m.2000     9
11  com.gopaktor.subscription.v2.premium.3m.2000    12
12      com.gopaktor.subscription.v12.premium.3m     8
13  com.gopaktor.subscription.v2.premium.6m.2000    14
14                  com.gopaktor.credit.v2.10000     1
15                   com.gopaktor.credit.v2.3000     2
16                   com.gopaktor.credit.v2.6000     3
17                   com.gopaktor.credit.v3.2000     4

In [28]:
product_map = uniq_prod.set_index('code')['product_id'].to_dict()

In [29]:
product_map

{0: u'com.gopaktor.credit.v2.1000',
 1: u'com.gopaktor.credit.v2.10000',
 2: u'com.gopaktor.credit.v2.3000',
 3: u'com.gopaktor.credit.v2.6000',
 4: u'com.gopaktor.credit.v3.2000',
 5: u'com.gopaktor.subscription.v1.premium.1m',
 6: u'com.gopaktor.subscription.v11.premium.6m',
 7: u'com.gopaktor.subscription.v12.premium.1m',
 8: u'com.gopaktor.subscription.v12.premium.3m',
 9: u'com.gopaktor.subscription.v2.premium.1m.2000',
 10: u'com.gopaktor.subscription.v2.premium.1m.3000',
 11: u'com.gopaktor.subscription.v2.premium.3m',
 12: u'com.gopaktor.subscription.v2.premium.3m.2000',
 13: u'com.gopaktor.subscription.v2.premium.6m',
 14: u'com.gopaktor.subscription.v2.premium.6m.2000',
 15: u'com.gopaktor.subscription.v3.premium.3m',
 16: u'com.gopaktor.subscription.v3.premium.6m',
 17: u'com.gopaktor.subscription.v4.premium.1m'}

In [30]:
product_map[1]

u'com.gopaktor.credit.v2.10000'

In [31]:
uniq_cust = pd.DataFrame(cust_prod.cust_id.unique())

In [32]:
uniq_cust.head(6)

0
0  3893506
1  3793254
2  6148794
3  6149220
4  6149432
5  6149434

In [33]:
uniq_cust.rename(columns={0:'cust_id'}, inplace=True)

In [34]:
uniq_cust.head()

cust_id
0  3893506
1  3793254
2  6148794
3  6149220
4  6149432

In [35]:
uniq_cust['cust_id'] = uniq_cust['cust_id'].astype('category')

In [36]:
uniq_cust['code'] = uniq_cust['cust_id'].cat.codes

In [37]:
uniq_cust.head(6)

cust_id  code
0  3893506     7
1  3793254     6
2  6148794    10
3  6149220    15
4  6149432    21
5  6149434    22

In [38]:
users_map = uniq_cust.set_index('code')['cust_id'].to_dict()

In [40]:
users_map[7]

u'3893506'

In [41]:
import json

In [42]:
json.dump(users_map, open("users_map.txt", 'w'))

In [43]:
json.dump(product_map, open("product_map.txt", 'w'))

In [44]:
%cat product_map.txt

{"0": "com.gopaktor.credit.v2.1000", "1": "com.gopaktor.credit.v2.10000", "2": "com.gopaktor.credit.v2.3000", "3": "com.gopaktor.credit.v2.6000", "4": "com.gopaktor.credit.v3.2000", "5": "com.gopaktor.subscription.v1.premium.1m", "6": "com.gopaktor.subscription.v11.premium.6m", "7": "com.gopaktor.subscription.v12.premium.1m", "8": "com.gopaktor.subscription.v12.premium.3m", "9": "com.gopaktor.subscription.v2.premium.1m.2000", "10": "com.gopaktor.subscription.v2.premium.1m.3000", "11": "com.gopaktor.subscription.v2.premium.3m", "12": "com.gopaktor.subscription.v2.premium.3m.2000", "13": "com.gopaktor.subscription.v2.premium.6m", "14": "com.gopaktor.subscription.v2.premium.6m.2000", "15": "com.gopaktor.subscription.v3.premium.3m", "16": "com.gopaktor.subscription.v3.premium.6m", "17": "com.gopaktor.subscription.v4.premium.1m"}

In [45]:
uniq_cust.to_csv('uniq_cust_cat', index=False, header=True)

In [46]:
uniq_prod.to_csv('uniq_prod_cat', index=False, header=True)